In [11]:
import numpy as np
from sklearn.model_selection import train_test_split

In [12]:
def sin(x, T=100):
    return np.sin(2.0 * np.pi * x / T)

def toy_problem(T=100, ampl=0.05):
    x = np.arange(0, 2*T +1)
    noise = ampl * np.random.uniform(low=-1.0, high=1.0, size=len(x))
    return sin(x) + noise

In [3]:
T = 100
f = toy_problem(T)

In [5]:
length_of_sequences = 2 * T
maxlen = 25

data = []
target = []

for i in range(0, length_of_sequences - maxlen + 1):
    data.append(f[i: i+maxlen])
    target.append(f[i+maxlen])

In [6]:
length_of_sequences = 2 * T
maxlen = 25

data = []
target = []

for i in range(0, length_of_sequences - maxlen + 1):
    data.append(f[i: i + maxlen])
    target.append(f[i + maxlen])

In [7]:
X = np.array(data).reshape(len(data), maxlen, 1)

Y = np.array(target).reshape(len(data), 1)

In [9]:
X = np.zeros((len(data), maxlen, 1), dtype=float)
Y = np.zeros((len(data), 1), dtype=float)

for i, seq in enumerate(data):
    for t, value in enumerate(seq):
        X[i, t, 0] = value
    Y[i, 0] = target[i]

In [13]:
N_train = int(len(data) * 0.9)
N_validation = len(data) - N_train

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=N_validation)

In [15]:
import tensorflow as tf

In [49]:
def inference(x, n_batch, maxlen=None, n_hidden=None, n_out=None):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)
    
    def bias_variable(shape):
        initial = tf.zeros(shape, dtype=tf.float32)
        return tf.Variable(initial)
    
    cell = tf.contrib.rnn.BasicRNNCell(n_hidden)
#     initial_state = cell.zero_state(n_batch, tf.float32)
    batch_size = tf.placeholder(tf.int32, [], name='batch_size')
    initial_state = cell.zero_state(batch_size, tf.float32)
    



    state = initial_state
    outputs = []
    with tf.variable_scope('RNN'):
        for t in range(maxlen):
            if t > 0:
                tf.get_variable_scope().reuse_variables()
                (cell_output, state) = cell(x[:, t, :], state)
                outputs.append(cell_output)
                
    output = output[-1]
    
    V = weight_variable([n_hidden, n_out])
    c = bias_variable([n_out])
    y = tf.matmul(output, V) + c
    
    return y

In [50]:
def loss(y, t):
    mse = tf.reduce_mean(tf.square(y - t))
    return mse

In [51]:
def training(loss):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999)
    
    train_step = optimizer.minimize(loss)
    return train_step

In [52]:
n_in = len(X[0][0])
n_hidden = 20
n_out = len(Y[0])

x = tf.placeholder(tf.float32, shape=[None, maxlen, n_in])
t = tf.placeholder(tf.float32, shape=[None, n_out])
n_batch = tf.placeholder(tf.float32)

y = inference(x, n_batch, maxlen=maxlen, n_hidden=n_hidden, n_out=n_out)
loss = loss(y, t)
train_step = training(loss)

ValueError: Variable RNN/basic_rnn_cell/kernel does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?